# OgmaJL - Julia Implementations of the Feynman Machine

## FocalMatrix - locally focussed weight matrix
The `FocalMatrix` is a weight matrix which connects an input tensor `ins` of size `isize` (`isize.x×isize.y×isize.z`) to an output tensor `outs` of size `osize` (`osize.x×osize.y×osize.z`), where each output receives input from units in an `x-y` window within `radius` units of the input unit "below" the `x-y` of the output unit.

Start with some imports to control RNG and display some types.

In [1]:
import Random
import Base.show

## Types

In [2]:
const AW = Float32
const XW = UInt32
const XIO = UInt16

UInt16

In [3]:
struct XIO3 x::XIO; y::XIO; z::XIO end

In [4]:
mutable struct XIO3M x::XIO; y::XIO; z::XIO end

In [5]:

show(io::IO, s::XIO3) = print(io, "XIO3[$(s.x)×$(s.y)×$(s.z)]")
show(io::IO, s::XIO3M) = show(io, XIO3(s.x, s.y, s.z))
show(io::IO, x::XW) = show(io, Int(x))
show(io::IO, x::XIO) = show(io, Int(x))

show (generic function with 237 methods)

In [6]:
s = XIO3(1,2,3)

XIO3[1×2×3]

In [7]:
sizeof(XIO3)

6

In [8]:
import Base.copy
copy(s::XIO3) = XIO3(s.x, s.y, s.z)

copy (generic function with 93 methods)

In [9]:
abstract type AbstractWeightMatrix end

struct FocalMatrix <: AbstractWeightMatrix
    isize::XIO3
    osize::XIO3
    radius::XIO
    xs::Array{XIO, 2}
    ys::Array{XIO, 2}
    idxs::Array{XW, 2}
    ws::Array{AW, 1}
    nWs::XW
end

In [10]:
function FocalMatrix(isize::XIO3, osize::XIO3, radius::Int)
    xs::Array{XIO, 2} = fill(1, (osize.x, 2))
    ys::Array{XIO, 2} = fill(1, (osize.y, 2))
    idxs::Array{XW, 2} = ones(XW, osize.x, osize.y)
    vc::XIO = 1

    outinx::Float32 = isize.x/osize.x
    outiny::Float32 = isize.y/osize.y
    for oy = 1:osize.y
        vc = 1 + floor(0.5 + (oy - 1) * outiny)
        ys[oy, :] = [max(1, vc - radius), min(isize.y , vc + radius)]
    end
    idx = 1
    for ox = 1:osize.x
        vc = 1 + floor(0.5 + (ox - 1) * outinx)
        xs[ox, :] = [max(1, vc - radius), min(isize.x, vc + radius)]
        dix = 1 + xs[ox, 2] - xs[ox, 1]
        for oy = 1:osize.y
            idxs[ox, oy] = idx
            diy = 1 + ys[oy, 2] - ys[oy, 1]
            cellCount = dix * diy * osize.z * isize.z
            idx += cellCount
        end
    end
    ws::Array{AW, 1} = rand(idx + 1)
    nWs::XW = idx
    FocalMatrix(isize, osize, radius, xs, ys, idxs, ws, nWs)
end

FocalMatrix

In [11]:
#=
    const xL ix = 8, iy = 10, iz = 3;
    const LPt isize{ix, iy, iz};
    const xL ox = 5, oy = 4, oz = 3;

=#
is = XIO3(8,10,3)
os = XIO3(5,4,3)
r = 3

3

In [12]:
Random.seed!(1234)
f = FocalMatrix(is, os, r)

FocalMatrix(XIO3[8×10×3], XIO3[5×4×3], 3, UInt16[1 4; 1 6; … ; 3 8; 4 8], UInt16[1 4; 1 7; 3 9; 6 10], UInt32[1 145 397 649; 829 1045 1423 1801; … ; 3520 3736 4114 4492; 4762 4942 5257 5572], Float32[0.59084463, 0.76679707, 0.5662374, 0.46008533, 0.7940257, 0.8541466, 0.20058604, 0.29861426, 0.24683718, 0.5796722  …  0.708213, 0.73846525, 0.9270241, 0.4111475, 0.48356026, 0.84817374, 0.11161669, 0.7702865, 0.6142066, 0.3792697], 5797)

In [13]:
println(f.ws[1:10])
println(f.xs)
println(f.ys)

Float32[0.59084463, 0.76679707, 0.5662374, 0.46008533, 0.7940257, 0.8541466, 0.20058604, 0.29861426, 0.24683718, 0.5796722]
UInt16[1 4; 1 6; 1 7; 3 8; 4 8]
UInt16[1 4; 1 7; 3 9; 6 10]


In [14]:
# using LinearAlgebra

In [15]:
function forward!(m::FocalMatrix, ins::Array{AW}, outs::Array{AW})
    idxW = 1
    local diz = m.isize.z
    local dix = 1; local diy = 1; local o = 0.0
    osize = m.osize
    isize = m.isize
    ws = m.ws
    xs, ys = m.xs, m.ys
    local bz = isize.z
    for ox::XIO in 1:osize.x
        ax, bx = xs[ox, :]
        dix = 1 + bx - ax
        for oy::XIO in 1:osize.y
            ay, by = ys[oy, :]
            diy = 1 + by - ay
            for oz::XIO in 1:osize.z
                o = 0.0
                iPos = 0
                for ix::XIO in ax:bx, iy::XIO in ay:by
                    for iz::XIO in 1:bz
                        o = o + ins[iPos + iz] * ws[idxW + iz]
                    end
                    idxW += diz
                    iPos += diz
                end
                oPos = oz + osize.z * (oy - 1) + (osize.z * osize.y) * (ox - 1)
                outs[oPos] = o
            end
        end
    end
    outs
end


forward! (generic function with 1 method)

In [16]:
# ins[10:12] .* f.ws[10:12]

In [17]:
function forward(m::FocalMatrix, ins::Array{AW}, outs::Array{AW})
    idxW = 1
    diz = m.isize.z
    dix = 1; diy = 1; o = 0.0
    osize = m.osize
    isize = m.isize
    #outs::Array{Float32} = fill(0.0, osize.x * osize.y * osize.z)
    ws = m.ws
    xs, ys = m.xs, m.ys
    bz = isize.z
    for ox::XIO in 1:osize.x
        ax, bx = xs[ox, :]
        dix = 1 + bx - ax
        for oy in 1:osize.y
            ay, by = ys[oy, :]
            diy = 1 + by - ay
            for oz in 1:osize.z
                o::AW = 0.0
                iPos = 0
                for ix in ax:bx, iy in ay:by
                    #println("ix $ix iy $iy iPos $iPos idxW $idxW")
                    for iz in 1:bz
                        o += ins[iPos + iz] * ws[idxW + iz]
                    end
                    idxW += diz
                    iPos += diz
                end
                oPos = oz + osize.z * (oy - 1) + (osize.z * osize.y) * (ox - 1)
                outs[oPos] = o
            end
        end
    end
    outs
end

forward (generic function with 1 method)

In [18]:
Random.seed!(56789)
ins = AW.(rand([-1,1], is.x * is.y * is.z) .* rand(is.x * is.y * is.z))
outs = zeros(AW, os.x * os.y * os.z)
forward!(f, ins, outs)

60-element Array{Float32,1}:
 2.9894378
 3.4524918
 5.414768
 1.9698108
 1.86852
 3.873893
 5.16832
 4.373421
 5.00158
 4.7478986
 3.362311
 2.8978314
 4.0864167
 ⋮
 5.1043787
 4.590091
 2.774628
 7.7735734
 2.5626953
 6.266735
 3.5568266
 4.9592505
 7.5303793
 4.5096736
 4.193823
 1.2336527

In [19]:
print(outs)

Float32[2.9894378, 3.4524918, 5.414768, 1.9698108, 1.86852, 3.873893, 5.16832, 4.373421, 5.00158, 4.7478986, 3.362311, 2.8978314, 4.0864167, 4.5952425, 2.9274929, 0.6543874, 1.3638287, 6.347465, 1.7846342, 3.4823189, 3.2542682, 2.941091, 5.2828045, 2.4702134, 5.261577, 3.263958, 2.257713, 2.4319875, 4.0329046, 2.281857, 3.47669, 2.5158207, 2.8616593, 3.9551666, 3.2946434, 6.26543, 2.722341, 2.0334377, 2.7777352, 8.503659, 1.1500036, 3.6527755, 4.189236, 3.0570748, 3.3649724, 2.880071, 6.247614, 5.2673626, 5.1043787, 4.590091, 2.774628, 7.7735734, 2.5626953, 6.266735, 3.5568266, 4.9592505, 7.5303793, 4.5096736, 4.193823, 1.2336527]

In [20]:
@code_warntype forward(f, ins, outs)

Variables
  #self#::Core.Compiler.Const(forward, false)
  m::FocalMatrix
  ins::Array{Float32,1}
  outs::Array{Float32,1}
  idxW::Int64
  diz::UInt16
  dix::Int64
  diy::Int64
  o::Float32
  osize::XIO3
  isize::XIO3
  ws::Array{Float32,1}
  xs::Array{UInt16,2}
  ys::Array{UInt16,2}
  bz::UInt16
  @_16::Union{Nothing, Tuple{Int64,Int64}}
  ox::UInt16
  ax::UInt16
  @_19::Int64
  bx::UInt16
  @_21::Union{Nothing, Tuple{Int64,Int64}}
  oy::Int64
  ay::UInt16
  @_24::Int64
  by::UInt16
  @_26::Union{Nothing, Tuple{Int64,Int64}}
  oz::Int64
  iPos::Int64
  @_29::Union{Nothing, Tuple{UInt16,UInt16}}
  oPos::Int64
  ix@_31::UInt16
  @_32::Union{Nothing, Tuple{UInt16,UInt16}}
  iy::UInt16
  @_34::Union{Nothing, Tuple{Int64,Int64}}
  ix@_35::UInt16
  iz::Int64

Body::Array{Float32,1}
1 ──        (idxW = 1)
│    %2   = Base.getproperty(m, :isize)::XIO3
│           (diz = Base.getproperty(%2, :z))
│           (dix = 1)
│           (diy = 1)
│    %6   = Base.convert(Main.AW, 0.0)::Core.Compiler.C

In [21]:
outs = zeros(AW, os.x * os.y * os.z)
forward(f, ins, outs)

60-element Array{Float32,1}:
 2.9894376
 3.4524927
 5.4147687
 1.9698106
 1.8685198
 3.8738937
 5.168321
 4.3734217
 5.0015807
 4.747899
 3.3623106
 2.897831
 4.086417
 ⋮
 5.104381
 4.5900908
 2.774628
 7.7735715
 2.5626953
 6.266736
 3.5568268
 4.959251
 7.530379
 4.5096745
 4.193823
 1.2336526

In [22]:
print(outs)

Float32[2.9894376, 3.4524927, 5.4147687, 1.9698106, 1.8685198, 3.8738937, 5.168321, 4.3734217, 5.0015807, 4.747899, 3.3623106, 2.897831, 4.086417, 4.595243, 2.9274924, 0.6543875, 1.363829, 6.3474665, 1.7846344, 3.4823186, 3.2542672, 2.9410915, 5.2828035, 2.4702127, 5.261576, 3.2639582, 2.2577138, 2.4319873, 4.032905, 2.2818565, 3.4766881, 2.515821, 2.8616586, 3.9551675, 3.2946434, 6.2654314, 2.722341, 2.0334377, 2.7777362, 8.503656, 1.1500053, 3.6527767, 4.1892343, 3.0570745, 3.3649707, 2.880071, 6.2476134, 5.267362, 5.104381, 4.5900908, 2.774628, 7.7735715, 2.5626953, 6.266736, 3.5568268, 4.959251, 7.530379, 4.5096745, 4.193823, 1.2336526]

In [23]:
using BenchmarkTools

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1260


In [24]:
@time forward!(f, ins, outs)

  0.000027 seconds (25 allocations: 2.344 KiB)


60-element Array{Float32,1}:
 2.9894378
 3.4524918
 5.414768
 1.9698108
 1.86852
 3.873893
 5.16832
 4.373421
 5.00158
 4.7478986
 3.362311
 2.8978314
 4.0864167
 ⋮
 5.1043787
 4.590091
 2.774628
 7.7735734
 2.5626953
 6.266735
 3.5568266
 4.9592505
 7.5303793
 4.5096736
 4.193823
 1.2336527

In [25]:
@benchmark forward!(f, ins, outs)

BenchmarkTools.Trial: 
  memory estimate:  2.34 KiB
  allocs estimate:  25
  --------------
  minimum time:     10.358 μs (0.00% GC)
  median time:      11.303 μs (0.00% GC)
  mean time:        12.655 μs (0.00% GC)
  maximum time:     56.388 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [26]:
@benchmark forward(f, ins, outs)

BenchmarkTools.Trial: 
  memory estimate:  2.34 KiB
  allocs estimate:  25
  --------------
  minimum time:     9.607 μs (0.00% GC)
  median time:      9.782 μs (0.00% GC)
  mean time:        9.943 μs (0.00% GC)
  maximum time:     22.896 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [27]:
@code_native forward!(f, ins, outs)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[15]:2 within `forward!'
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	subq	$344, %rsp              ## imm = 0x158
	movq	%rsi, %r15
	vxorpd	%xmm0, %xmm0, %xmm0
	vmovapd	%xmm0, -112(%rbp)
	movq	$0, -96(%rbp)
	movq	%rsi, -376(%rbp)
	movabsq	$jl_get_ptls_states_fast, %rax
	callq	*%rax
	movq	$4, -112(%rbp)
	movq	(%rax), %rcx
	movq	%rcx, -104(%rbp)
	leaq	-112(%rbp), %rcx
	movq	%rax, -200(%rbp)
	movq	%rcx, (%rax)
	movq	(%r15), %r8
	movq	16(%r15), %rax
; │ @ In[15]:10 within `forward!'
; │┌ @ range.jl:3 within `Colon'
; ││┌ @ promotion.jl:282 within `promote'
; │││┌ @ promotion.jl:259 within `_promote'
; ││││┌ @ number.jl:7 within `convert'
; │││││┌ @ boot.jl:707 within `Int64'
; ││││││┌ @ boot.jl:630 within `toInt64'
	movzwl	6(%r8), %ecx
	movq	%rcx, -224(%rbp)
	testq	%rcx, %rcx
; │└└└└└└
	je	L1424
	movq	%rax, -176(%rbp)
	movq	8(%r15), %rax
	movq	%rax, -168(%rbp)
	movq	40(%r8), %rax
	movq	%ra

In [28]:
using Profile

In [29]:
@profile forward(f, ins, outs)

60-element Array{Float32,1}:
 2.9894376
 3.4524927
 5.4147687
 1.9698106
 1.8685198
 3.8738937
 5.168321
 4.3734217
 5.0015807
 4.747899
 3.3623106
 2.897831
 4.086417
 ⋮
 5.104381
 4.5900908
 2.774628
 7.7735715
 2.5626953
 6.266736
 3.5568268
 4.959251
 7.530379
 4.5096745
 4.193823
 1.2336526

In [30]:
Profile.print()

Overhead ╎ [+additional indent] Count File:Line; Function
 ╎5 @Base/task.jl:358; (::IJulia.var"#15#18")()
 ╎ 5 @IJulia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
 ╎  5 @Base/essentials.jl:711; invokelatest
 ╎   5 @Base/essentials.jl:712; #invokelatest#1
 ╎    5 ...rc/execute_request.jl:67; execute_request(::ZMQ.Socket, ::I...
 ╎     5 ...c/SoftGlobalScope.jl:218; softscope_include_string(::Modu...
4╎    ╎ 5 @Base/boot.jl:331; eval
Total snapshots: 5


In [ ]:
using PkgTemplates

In [ ]:
t = Template(;
    dir="~/src/ogmajl",
    plugins=[
        Git(; manifest=true, ssh=true),
        Codecov(),
        TravisCI(; x86=true),
        Documenter{TravisCI}(),
    ],
)

In [ ]:
# t("OgmaJL")

In [31]:
function locateWeight(m::FocalMatrix, index::XW) 
    o::XIO3M = XIO3M(1, 1, 1)
    i::XIO3M = XIO3M(1, 1, 1)
    i.z = index % m.isize.z;
    index = 1 + div(index, m.isize.z)
    idx::XW = 1
    for o.x = 1:m.osize.x
        dix::XIO = 1 + m.xs[o.x, 2] - m.xs[o.x, 1]
        for o.y = 1:m.osize.y
            diy::XIO = 1 + m.ys[o.y, 2] - m.ys[o.y, 1]
            dixy::XIO = dix * diy
            cellCount::XW = dixy * m.osize.z
            if index < idx + cellCount
                index -= idx
                o.z = 1 + div(index, dixy)
                index = index % dixy
                i.y = m.ys[o.y, 1] + (index % diy)
                i.x = m.xs[o.x, 1] + div(index, diy)
                return XIO3(i.x, i.y, i.z), XIO3(o.x, o.y, o.z)
            end
            idx += cellCount
        end
    end
    return (i.x, i.y, i.z), (o.x, o.y, o.z)
end


locateWeight (generic function with 1 method)

In [32]:
locateWeight(m::FocalMatrix, index::Int) = locateWeight(m, XW(index))

locateWeight (generic function with 2 methods)

In [33]:
i, o = locateWeight(f, 4142)

(XIO3[4×5×2], XIO3[4×3×1])

In [34]:
@benchmark i, o = locateWeight(f, 4142)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     72.431 ns (0.00% GC)
  median time:      72.695 ns (0.00% GC)
  mean time:        77.480 ns (0.00% GC)
  maximum time:     212.205 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     973

In [35]:
function sparse(m::FocalMatrix, outs::Array{AW}) 
    sparse = zero.(outs)
    tops = ones(XIO, m.osize.x * m.osize.y)
    i::XW = 1
    for ox::XIO = 1:m.osize.x, oy = 1:m.osize.y
        top = outs[i]
        itop = i
        i += 1
        for oz = 2:m.osize.z
            if outs[i] > top
                top = outs[i]
                itop = i
                tops[oy + (ox - 1) * m.osize.y] = oz
            end
            i += 1
        end
        sparse[itop] = top
    end
    return sparse, tops
end


sparse (generic function with 1 method)

In [36]:
# x = ones(AW, 10)
outs = zeros(AW, os.x * os.y * os.z)
forward(f, ins, outs)

60-element Array{Float32,1}:
 2.9894376
 3.4524927
 5.4147687
 1.9698106
 1.8685198
 3.8738937
 5.168321
 4.3734217
 5.0015807
 4.747899
 3.3623106
 2.897831
 4.086417
 ⋮
 5.104381
 4.5900908
 2.774628
 7.7735715
 2.5626953
 6.266736
 3.5568268
 4.959251
 7.530379
 4.5096745
 4.193823
 1.2336526

In [37]:
sparseouts, tops = sparse(f, outs)

(Float32[0.0, 0.0, 5.4147687, 0.0, 0.0, 3.8738937, 5.168321, 0.0, 0.0, 4.747899  …  0.0, 7.7735715, 0.0, 0.0, 0.0, 0.0, 7.530379, 4.5096745, 0.0, 0.0], UInt16[3, 3, 1, 1, 2, 3, 2, 2, 1, 2, 1, 3, 3, 1, 1, 2, 1, 1, 3, 1])

In [38]:
println(sparseouts)
println(Int.(tops))

Float32[0.0, 0.0, 5.4147687, 0.0, 0.0, 3.8738937, 5.168321, 0.0, 0.0, 4.747899, 0.0, 0.0, 0.0, 4.595243, 0.0, 0.0, 0.0, 6.3474665, 0.0, 3.4823186, 0.0, 0.0, 5.2828035, 0.0, 5.261576, 0.0, 0.0, 0.0, 4.032905, 0.0, 3.4766881, 0.0, 0.0, 0.0, 0.0, 6.2654314, 0.0, 0.0, 2.7777362, 8.503656, 0.0, 0.0, 4.1892343, 0.0, 0.0, 0.0, 6.2476134, 0.0, 5.104381, 0.0, 0.0, 7.7735715, 0.0, 0.0, 0.0, 0.0, 7.530379, 4.5096745, 0.0, 0.0]
[3, 3, 1, 1, 2, 3, 2, 2, 1, 2, 1, 3, 3, 1, 1, 2, 1, 1, 3, 1]


In [39]:
import Random # hide
Random.seed!(1234) # hide
f1 = FocalMatrix(XIO3(3, 4, 3), XIO3(4, 5, 3), 1)

FocalMatrix(XIO3[3×4×3], XIO3[4×5×3], 1, UInt16[1 2; 1 3; 2 3; 2 3], UInt16[1 2; 1 3; … ; 2 4; 3 4], UInt32[1 37 … 145 199; 235 289 … 451 532; 586 622 … 730 784; 820 856 … 964 1018], Float32[0.59084463, 0.76679707, 0.5662374, 0.46008533, 0.7940257, 0.8541466, 0.20058604, 0.29861426, 0.24683718, 0.5796722  …  0.587723, 0.42177233, 0.4241981, 0.3947768, 0.6542789, 0.11622245, 0.7428318, 0.48858702, 0.36625502, 0.7665335], 1054)

In [40]:
Random.seed!(1234) # hide
m = FocalMatrix(XIO3(3, 4, 3), XIO3(4, 5, 3), 1)
Random.seed!(56789) # hide
ins = AW.(rand([-1,1], 3 * 4 * 3) .* rand(3 * 4 * 3))
outs = zeros(AW, 4 * 5 * 3)

forward!(m, ins, outs)

60-element Array{Float32,1}:
  0.2664309
  1.8382773
 -0.067347586
  1.5660199
  0.26483107
  2.2840357
  0.71645874
  1.9513111
  2.453909
  0.20646897
  0.8714455
  2.0525088
  0.02235264
  ⋮
  2.767889
  1.1482695
  1.3052162
  1.0694035
  3.3695157
  1.2626712
  2.4824812
  0.7995742
  0.70120615
  0.70984924
  1.0287566
  0.99003124

In [41]:
abstract type AbstractEncoder end

struct SparseFocalEncoder <: AbstractEncoder
    inputs::Vector{Vector{AW}}
    outputs::Vector{AW}
    weight_tensors::Vector{FocalMatrix}
end

In [42]:
function SparseFocalEncoder(isizes::Vector{XIO3}, osize::XIO3, radii::Vector{Int})
    weight_tensors = [FocalMatrix(isizes[i], osize, radii[i]) for i in 1:length(isizes)]
    inputs = [zeros(AW, isize.x * isize.y * isize.z) for isize in isizes]
    outputs = zeros(AW, osize.x * osize.y * osize.z)
    SparseFocalEncoder(inputs, outputs, weight_tensors)
end

SparseFocalEncoder

In [43]:
f1 = FocalMatrix(XIO3(3, 4, 3), XIO3(4, 5, 3), 1)

FocalMatrix(XIO3[3×4×3], XIO3[4×5×3], 1, UInt16[1 2; 1 3; 2 3; 2 3], UInt16[1 2; 1 3; … ; 2 4; 3 4], UInt32[1 37 … 145 199; 235 289 … 451 532; 586 622 … 730 784; 820 856 … 964 1018], Float32[0.43374178, 0.39928582, 0.67318344, 0.33292052, 0.92558473, 0.6088412, 0.78991354, 0.8466604, 0.37948284, 0.6476723  …  0.5636447, 0.82134634, 0.6928938, 0.7782518, 0.23702739, 0.23547582, 0.86335, 0.20046139, 0.1435705, 0.3372616], 1054)

In [44]:
Random.seed!(34567)
osize = XIO3(4, 5, 3)
isizes = [XIO3(3, 4, 3), XIO3(5, 3, 2)]
e = SparseFocalEncoder(isizes, osize, [1, 1])

SparseFocalEncoder(Array{Float32,1}[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], FocalMatrix[FocalMatrix(XIO3[3×4×3], XIO3[4×5×3], 1, UInt16[1 2; 1 3; 2 3; 2 3], UInt16[1 2; 1 3; … ; 2 4; 3 4], UInt32[1 37 … 145 199; 235 289 … 451 532; 586 622 … 730 784; 820 856 … 964 1018], Float32[0.09329348, 0.19942068, 0.15421993, 0.09267315, 0.44426835, 0.28764495, 0.8766207, 0.53800136, 0.92338425, 0.006675155  …  0.64751536, 0.21078563, 0.96265614, 0.53268325, 0.6183985, 0.5579505, 0.53462726, 0.96415246, 0.98463225, 0.91976595], 1054), FocalMatrix(XIO3[5×3×2], XIO3[4×5×3], 1, UInt16[1 2; 1 3; 3 5; 4 5], UInt16[1 2; 1 3; … ; 2 3; 2 3], UInt32[1 25 … 97 121; 145 181 … 289 325; 361 397 … 505 541; 577 601 … 673 697], Float32[

In [45]:
zero_array(T::Type, s::XIO3) = zeros(T, s.x * s.y * s.z)
rand_array(T::Type, s::XIO3) = AW.(rand([-1,1], s.x * s.y * s.z) .* rand(s.x * s.y * s.z))

rand_array (generic function with 1 method)

In [46]:
function forward!(e::SparseFocalEncoder)
    for i in 1:length(e.weight_tensors)
        forward!(e.weight_tensors[i], ins[i], e.outputs)
    end
    e.outputs
end

forward! (generic function with 2 methods)

In [47]:
Random.seed!(89123)
outs = zero_array(AW, osize)
# outs = -1 .+ (2 .* rand(length(outs)))
ins = [rand_array(AW, isize) for isize in isizes]

2-element Array{Array{Float32,1},1}:
 [-0.7364922, -0.7875477, -0.219467, 0.25228944, 0.95381874, -0.7674291, -0.96530443, 0.86926484, -0.4243301, 0.22602618  …  -0.6403681, 0.5357497, -0.9401945, -0.73612064, 0.34874922, -0.19932969, 0.13570413, 0.120193616, 0.76010996, -0.13946737]
 [-0.47559175, 0.670355, 0.77992654, 0.927623, 0.4101152, -0.7436763, -0.49791625, 0.21972069, -0.71047175, -0.94052833  …  0.11339265, 0.7266594, -0.119111694, -0.102372624, -0.50460494, 0.4063471, -0.53905845, 0.72128356, 0.54966044, -0.83017486]

In [48]:
e.inputs .= ins; e.outputs .= outs
forward!(e)

60-element Array{Float32,1}:
  0.7514992
  0.7809855
  0.8526475
  1.370517
 -0.18526773
 -0.08679871
 -0.43857747
  0.04915623
  1.158513
  1.0250347
 -1.1880852
  0.57661813
  0.7848313
  ⋮
 -0.53117216
  0.355751
  0.49985623
 -0.61717725
  0.05028704
 -0.94615906
  0.8104348
  1.3214023
  0.55635864
  0.4760884
  1.1940714
  1.6242076

In [49]:
println(e.outputs)

Float32[0.7514992, 0.7809855, 0.8526475, 1.370517, -0.18526773, -0.08679871, -0.43857747, 0.04915623, 1.158513, 1.0250347, -1.1880852, 0.57661813, 0.7848313, 0.43143368, 0.29488307, -0.48093575, 1.8385435, -0.13626836, 1.7437478, -0.87667084, 0.9475402, -0.013112253, 0.35094848, 0.31001124, 0.04208443, 0.09024589, 0.0022133125, 0.5136844, 1.7716193, 0.51258564, 0.054811314, 0.55999625, -0.50662094, -0.04656465, -0.09488313, 1.1188865, -1.050364, -0.779522, 0.5552015, 0.029693635, 0.1861073, -0.20155588, -0.0623705, 0.37859172, 0.35813853, 0.2096517, 0.85903984, 0.6273687, -0.53117216, 0.355751, 0.49985623, -0.61717725, 0.05028704, -0.94615906, 0.8104348, 1.3214023, 0.55635864, 0.4760884, 1.1940714, 1.6242076]


In [50]:
function sparse(osize::XIO3, outs::Array{AW})
    sparse = zero.(outs)
    tops = ones(XIO, osize.x * osize.y)
    i::XW = 1
    for ox::XIO = 1:osize.x, oy = 1:osize.y
        top = outs[i]
        itop = i
        i += 1
        for oz = 2:osize.z
            if outs[i] > top
                top = outs[i]
                itop = i
                tops[oy + (ox - 1) * osize.y] = oz
            end
            i += 1
        end
        sparse[itop] = top
    end
    return sparse, tops
end


sparse (generic function with 2 methods)

In [51]:
sparseouts, tops = sparse(osize, e.outputs)

(Float32[0.0, 0.0, 0.8526475, 1.370517, 0.0, 0.0, 0.0, 0.0, 1.158513, 1.0250347  …  0.49985623, 0.0, 0.05028704, 0.0, 0.0, 1.3214023, 0.0, 0.0, 0.0, 1.6242076], UInt16[3, 1, 3, 1, 1, 2, 1, 2, 2, 2, 2, 3, 3, 2, 2, 2, 3, 2, 2, 3])

In [52]:
println(sparseouts)
println(tops)

Float32[0.0, 0.0, 0.8526475, 1.370517, 0.0, 0.0, 0.0, 0.0, 1.158513, 1.0250347, 0.0, 0.0, 0.7848313, 0.0, 0.0, 0.0, 1.8385435, 0.0, 1.7437478, 0.0, 0.0, 0.0, 0.35094848, 0.0, 0.0, 0.09024589, 0.0, 0.0, 1.7716193, 0.0, 0.0, 0.55999625, 0.0, 0.0, 0.0, 1.1188865, 0.0, 0.0, 0.5552015, 0.0, 0.1861073, 0.0, 0.0, 0.37859172, 0.0, 0.0, 0.85903984, 0.0, 0.0, 0.0, 0.49985623, 0.0, 0.05028704, 0.0, 0.0, 1.3214023, 0.0, 0.0, 0.0, 1.6242076]
UInt16[3, 1, 3, 1, 1, 2, 1, 2, 2, 2, 2, 3, 3, 2, 2, 2, 3, 2, 2, 3]


In [53]:
rand_array(T::Type, s::XIO3) = AW.(rand([-1,1], s.x * s.y * s.z) .* rand(s.x * s.y * s.z))

rand_array (generic function with 1 method)

In [54]:
Random.seed!(1234) # hide
osize = XIO3(4, 5, 3)
outs = rand_array(AW, osize)
sparseouts, tops = sparse(osize, outs)

(Float32[0.0, 0.0, 0.5250573, 0.6120098, 0.0, 0.0, 0.0, 0.576082, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.982502, 0.4270338, 0.0, 0.0, 0.5586466, 0.0, 0.0], UInt16[3, 1, 2, 3, 3, 2, 2, 3, 1, 3, 1, 1, 3, 3, 1, 3, 1, 3, 1, 1])